

<font face="Lucida Handwriting
" size=6 color='#CDB79E'> Libraries import <font>


In [3]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

<font face="Lucida Handwriting
" size=6 color='#CDB79E'>Data Collection Porto <font>


In [ ]:
# getting "RENT" listings in Porto from Remax Portogul (https://www.remax.pt/en/)

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 17 #340 houses for rent, last page is 17 (340/20 = 17)
pages = list(range(1, last_page+1))

url = "https://www.remax.pt/en/rent?searchQueryState=%7B%22regionName%22:%22Porto%22,%22businessType%22:2,%22page%22:1,%22regionID%22:%2278%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.21011650909249,%22longitude%22:-8.449367332491727%7D,%22regionZoom%22:9,%22prn%22:%22Porto%22%7D"


for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_rent = pd.DataFrame(property_list)
df_rent.to_csv(r'porto_district_rent.csv', index = False, header=True) # saving as csv

In [11]:
porto_rent = pd.read_csv('porto_district_rent.csv')   

In [12]:
porto_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      340 non-null    object
 1   Address    340 non-null    object
 2   Prop_type  340 non-null    object
 3   Size       340 non-null    object
 4   Bath       340 non-null    int64 
 5   Bed        340 non-null    object
dtypes: int64(1), object(5)
memory usage: 16.1+ KB


In [13]:
porto_rent.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,1 800 € / Monthly,"Porto - Porto, Cedofeita, Santo Ildefonso, ...",Condo/Apartment,66 m2,1,1
1,800 € / Monthly,"Porto - Porto, Cedofeita, Santo Ildefonso, ...",Condo/Apartment,40 m2,1,0
2,900 € / Monthly,"Porto - Porto, Cedofeita, Santo Ildefonso, ...",Condo/Apartment,40 m2,1,0
3,800 € / Monthly,"Porto - Porto, Cedofeita, Santo Ildefonso, ...",Condo/Apartment,40 m2,1,0
4,21 000 € / Monthly,"Porto - Porto, Aldoar, Foz do Douro e Nevog...",Business Office,1220 m2,0,--


In [ ]:
# code for getting "BUY" listings - HOUSES
# found out page only goes to 100, so cannot scrap all at once => HOUSES only

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 65 #1288 houses for buy, last page is 65 (1288/20 = 64.4)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Porto%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2278%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.21011650909249,%22longitude%22:-8.449367332491727%7D,%22regionZoom%22:9,%22prn%22:%22Porto%22,%22listingClass%22:1,%22mapScroll%22:false,%22listingTypes%22:%5B%2211%22%5D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_only_houses= pd.DataFrame(property_list)

In [ ]:
# code for getting "BUY" listings - CONDOS/APTS part 1
# page only goes to 100, so cannot scrap all at once
# 0 < price < 250,000 - 1011 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 51 #1020 houses for rent, last page is 51 (1020/20 = 51)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Porto%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2278%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.21011650909249,%22longitude%22:-8.449367332491727%7D,%22regionZoom%22:9,%22prn%22:%22Porto%22,%22listingClass%22:1,%22mapScroll%22:false,%22listingTypes%22:%5B%221%22%5D,%22price%22:%7B%22min%22:0,%22max%22:250000%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Con_Aprt_lessThan250 = pd.DataFrame(property_list)


In [ ]:
# code for getting for buy listings - CONDOS/APTS part 2
# page only goes to 100, so cannot scrap all at once
# 250,000 < price < 500,000 - 534 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 24 #534 houses for rent, last page is 27 (534/20 = 26.7)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Porto%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2278%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.21011650909249,%22longitude%22:-8.449367332491727%7D,%22regionZoom%22:9,%22prn%22:%22Porto%22,%22listingClass%22:1,%22mapScroll%22:false,%22listingTypes%22:%5B%221%22%5D,%22price%22:%7B%22min%22:250001,%22max%22:500000%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Con_Aprt_250K_500K = pd.DataFrame(property_list)


In [ ]:
# code for getting for buy listings - CONDS/APTS part 3
# page only goes to 100, so cannot scrap all at once
# price > 500,000 - 248 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 13 #248 houses for rent, last page is 13 (248/20 = 12.4)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Porto%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2278%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.21011650909249,%22longitude%22:-8.449367332491727%7D,%22regionZoom%22:9,%22prn%22:%22Porto%22,%22listingClass%22:1,%22mapScroll%22:false,%22listingTypes%22:%5B%221%22%5D,%22price%22:%7B%22min%22:500001,%22max%22:null%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Aprt_Above_500K = pd.DataFrame(property_list)


<font face="Lucida Handwriting
" size=4 color='#CDB79E'> Connecting all the data into one 
    CSV file <font>


In [ ]:
frames = [df_buy_only_houses, df_buy_Con_Aprt_lessThan250,df_buy_Con_Aprt_250K_500K, df_buy_Aprt_Above_500K ]
df_buy = pd.concat(frames)
df_buy.to_csv(r'Porto_Buy.csv', index = False, header=True) #saving as csv

In [25]:
data_Porto = pd.read_csv('Porto_Buy.csv')   

In [16]:
data_Porto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      3040 non-null   object
 1   Address    3040 non-null   object
 2   Prop_type  3040 non-null   object
 3   Size       3040 non-null   object
 4   Bath       3040 non-null   int64 
 5   Bed        3040 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 142.6+ KB


In [17]:
data.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,350 000 €,"Porto - Maia, Castêlo da Maia",House,233 m2,4,4
1,98 000 €,"Porto - Paredes, Lordelo",House,60 m2,1,2
2,300 000 €,"Porto - Felgueiras, Vila Fria e Vizela (São...",House,642 m2,3,4
3,199 900 €,"Porto - Amarante, Telões",House,188 m2,3,3
4,235 000 €,"Porto - Marco de Canaveses, Tabuado",House,200 m2,2,3


<font face="Lucida Handwriting
" size=6 color='#CDB79E'>Data Collection Braga <font>


In [8]:
# getting "RENT" listings in Braga from Remax Portogul (https://www.remax.pt/en/)

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 9 #169 houses for rent, last page is 9 (169/20 = 8.45)
pages = list(range(1, last_page+1))

url = "https://www.remax.pt/en/rent?searchQueryState=%7B%22regionName%22:%22Braga%22,%22businessType%22:2,%22page%22:1,%22regionID%22:%2258%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.50524075469569,%22longitude%22:-8.41451350539216%7D,%22regionZoom%22:9,%22prn%22:%22Braga%22%7D"

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_rent = pd.DataFrame(property_list)
df_rent.to_csv(r'braga_district_rent.csv', index = False, header=True) # saving as csv


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 7.37MB/s]
C:\Users\roych\AppData\Local\Temp\ipykernel_16252\3394645309.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
df_rent.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,650 € / Monthly,"Braga - Braga, São Victor",Store,120 m2,1,--
1,500 € / Monthly,"Braga - Braga, Braga (Maximinos, Sé e Civid...",Condo/Apartment,25 m2,1,0
2,250 € / Monthly,"Braga - Braga, Braga (Maximinos, Sé e Civid...",Business Office,42 m2,1,0
3,600 € / Monthly,"Braga - Póvoa de Lanhoso, Rendufinho",Farm,0 m2,0,--
4,650 € / Monthly,"Braga - Guimarães, Oliveira, São Paio e São...",Condo/Apartment,63 m2,1,1


In [14]:
# code for getting "BUY" listings - HOUSES // BRAGA
# found out page only goes to 100, so cannot scrap all at once => HOUSES only

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 62 #1225 houses for buy, last page is 62 (1225/20 = 61.5)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Braga%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2258%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.50524075469569,%22longitude%22:-8.41451350539216%7D,%22regionZoom%22:9,%22prn%22:%22Braga%22,%22listingClass%22:1,%22listingTypes%22:%5B%2211%22%5D,%22mapScroll%22:false%7D'
for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_only_houses_Braga= pd.DataFrame(property_list)

C:\Users\roych\AppData\Local\Temp\ipykernel_16252\2793061713.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
df_buy_only_houses_Braga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1240 non-null   object
 1   Address    1240 non-null   object
 2   Prop_type  1240 non-null   object
 3   Size       1240 non-null   object
 4   Bath       1240 non-null   object
 5   Bed        1240 non-null   object
dtypes: object(6)
memory usage: 58.2+ KB


In [19]:
df_buy_only_houses_Braga.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,319 000 €,"Braga - Guimarães, Airão Santa Maria, Airão...",House,339 m2,0,3
1,439 000 €,"Braga - Esposende, Apúlia e Fão",House,274 m2,0,2
2,250 000 €,"Braga - Póvoa de Lanhoso, Póvoa de Lanhoso ...",House,198 m2,1,4
3,315 000 €,"Braga - Vila Verde, Soutelo",House,208 m2,3,3
4,269 000 €,"Braga - Vila Nova de Famalicão, Avidos e La...",House,382 m2,3,3


In [20]:
# code for getting "BUY" listings - CONDOS/APTS part 1 // Braga
# page only goes to 100, so cannot scrap all at once
# 0 < price < 250,000 - 661 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 31 #661 houses for rent, last page is 31 (661/20 = 30.05)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Braga%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2258%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.50524075469569,%22longitude%22:-8.41451350539216%7D,%22regionZoom%22:9,%22prn%22:%22Braga%22,%22listingClass%22:1,%22listingTypes%22:%5B%221%22%5D,%22mapScroll%22:false,%22price%22:%7B%22min%22:0,%22max%22:250000%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Con_Aprt_lessThan250_Braga = pd.DataFrame(property_list)


C:\Users\roych\AppData\Local\Temp\ipykernel_16252\2072665191.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [21]:
df_buy_Con_Aprt_lessThan250_Braga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      620 non-null    object
 1   Address    620 non-null    object
 2   Prop_type  620 non-null    object
 3   Size       620 non-null    object
 4   Bath       620 non-null    object
 5   Bed        620 non-null    object
dtypes: object(6)
memory usage: 29.2+ KB


In [22]:
df_buy_Con_Aprt_lessThan250_Braga.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,240 000 €,"Braga - Guimarães, Azurém",Condo/Apartment,130 m2,2,3
1,154 900 €,"Braga - Braga, São Victor",Condo/Apartment,73 m2,1,1
2,169 000 €,"Braga - Póvoa de Lanhoso, Póvoa de Lanhoso ...",Condo/Apartment,130 m2,2,3
3,150 000 €,"Braga - Vieira do Minho, Vieira do Minho",Condo/Apartment,133 m2,2,3
4,210 000 €,"Braga - Guimarães, Creixomil",Condo/Apartment,136 m2,3,4


In [23]:
# code for getting for buy listings - CONDOS/APTS part 2 //Braga
# page only goes to 100, so cannot scrap all at once
# 250,000 < price < 500,000 - 211 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 11 #211 houses for rent, last page is 11 (211/20 = 10.55)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Braga%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2258%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.50524075469569,%22longitude%22:-8.41451350539216%7D,%22regionZoom%22:9,%22prn%22:%22Braga%22,%22listingClass%22:1,%22listingTypes%22:%5B%221%22%5D,%22mapScroll%22:false,%22price%22:%7B%22min%22:250001,%22max%22:500000%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Con_Aprt_250K_500K_Braga = pd.DataFrame(property_list)


C:\Users\roych\AppData\Local\Temp\ipykernel_16252\1955907597.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [24]:
df_buy_Con_Aprt_250K_500K_Braga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      220 non-null    object
 1   Address    220 non-null    object
 2   Prop_type  220 non-null    object
 3   Size       220 non-null    object
 4   Bath       220 non-null    object
 5   Bed        220 non-null    object
dtypes: object(6)
memory usage: 10.4+ KB


In [25]:
df_buy_Con_Aprt_250K_500K_Braga.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,325 000 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,200 m2,3,3
1,325 000 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,250 m2,3,3
2,325 000 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,250 m2,3,3
3,325 000 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,200 m2,3,3
4,325 000 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,200 m2,3,3


In [26]:
# code for getting for buy listings - CONDS/APTS part 3 // Braga
# page only goes to 100, so cannot scrap all at once
# price > 500,000 - 22 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 2 #22 houses for rent, last page is 2 (22/20 = 1.)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Braga%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2258%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:41.50524075469569,%22longitude%22:-8.41451350539216%7D,%22regionZoom%22:9,%22prn%22:%22Braga%22,%22listingClass%22:1,%22listingTypes%22:%5B%221%22%5D,%22mapScroll%22:false,%22price%22:%7B%22min%22:500001,%22max%22:null%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Aprt_Above_500K_Braga = pd.DataFrame(property_list)


C:\Users\roych\AppData\Local\Temp\ipykernel_16252\812928494.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [27]:
df_buy_Aprt_Above_500K_Braga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      40 non-null     object
 1   Address    40 non-null     object
 2   Prop_type  40 non-null     object
 3   Size       40 non-null     object
 4   Bath       40 non-null     object
 5   Bed        40 non-null     object
dtypes: object(6)
memory usage: 2.0+ KB


In [28]:
df_buy_Aprt_Above_500K_Braga.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,1 550 000 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,282 m2,3,4
1,1 300 000 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,282 m2,3,4
2,501 500 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,134 m2,3,3
3,548 500 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,128 m2,3,3
4,548 500 €,"Braga - Braga, Nogueira, Fraião e Lamaçães",Condo/Apartment,135 m2,3,3


<font face="Lucida Handwriting
" size=4 color='#CDB79E'> Connecting all the data into one 
    CSV file <font>


In [30]:
frames_Braga = [df_buy_only_houses_Braga, df_buy_Con_Aprt_lessThan250_Braga,df_buy_Con_Aprt_250K_500K_Braga, df_buy_Aprt_Above_500K_Braga ]
df_buy_Braga = pd.concat(frames_Braga)
df_buy_Braga.to_csv(r'Braga_Buy.csv', index = False, header=True) #saving as csv

In [26]:
data_Braga = pd.read_csv('Braga_Buy.csv')

In [32]:
df_buy_Braga.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2120 entries, 0 to 39
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      2120 non-null   object
 1   Address    2120 non-null   object
 2   Prop_type  2120 non-null   object
 3   Size       2120 non-null   object
 4   Bath       2120 non-null   object
 5   Bed        2120 non-null   object
dtypes: object(6)
memory usage: 115.9+ KB


In [33]:
df_buy_Braga.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,319 000 €,"Braga - Guimarães, Airão Santa Maria, Airão...",House,339 m2,0,3
1,439 000 €,"Braga - Esposende, Apúlia e Fão",House,274 m2,0,2
2,250 000 €,"Braga - Póvoa de Lanhoso, Póvoa de Lanhoso ...",House,198 m2,1,4
3,315 000 €,"Braga - Vila Verde, Soutelo",House,208 m2,3,3
4,269 000 €,"Braga - Vila Nova de Famalicão, Avidos e La...",House,382 m2,3,3


<font face="Lucida Handwriting
" size=6 color='#CDB79E'>Data Collection Lisboa <font>

In [4]:
# getting "RENT" listings in Lisboa from Remax Portogul (https://www.remax.pt/en/)

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 46 #907 houses for rent, last page is 46 (907/20 = 45.35)
pages = list(range(1, last_page+1))

url = "https://www.remax.pt/en/rent?searchQueryState=%7B%22regionName%22:%22Lisboa%20%22,%22businessType%22:2,%22page%22:1,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22prn%22:%22Lisboa%20%22,%22mapScroll%22:false%7D"
for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_rent = pd.DataFrame(property_list)
df_rent.to_csv(r'lisboa_district_rent.csv', index = False, header=True) # saving as csv


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 12.2MB/s]
C:\Users\roych\AppData\Local\Temp\ipykernel_7852\2798737085.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
lisboa_rent = pd.read_csv('lisboa_district_rent.csv')   

In [6]:
lisboa_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      920 non-null    object
 1   Address    920 non-null    object
 2   Prop_type  920 non-null    object
 3   Size       920 non-null    object
 4   Bath       920 non-null    int64 
 5   Bed        920 non-null    object
dtypes: int64(1), object(5)
memory usage: 43.2+ KB


In [7]:
lisboa_rent.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,950 € / Monthly,"Lisboa - Cascais, Cascais e Estoril",Condo/Apartment,57 m2,1,1
1,1 900 € / Monthly,"Lisboa - Cascais, São Domingos de Rana",Condo/Apartment,139 m2,2,3
2,1 900 € / Monthly,"Lisboa - Cascais, São Domingos de Rana",Condo/Apartment,139 m2,2,3
3,850 € / Monthly,"Lisboa - Lisboa, Olivais",Condo/Apartment,55 m2,1,2
4,2 350 € / Monthly,"Lisboa - Lisboa, Arroios",Condo/Apartment,112 m2,2,2


In [8]:
# code for getting "BUY" listings - HOUSES Lisboa
# found out page only goes to 100, so cannot scrap all at once => HOUSES only

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 76 #1288 houses for buy, last page is 1,517 (1,517/20 = 75.85)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Lisboa%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2276%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:38.8404598668955,%22longitude%22:-9.2186952991834%7D,%22regionZoom%22:9,%22prn%22:%22Lisboa%22,%22listingClass%22:1,%22listingTypes%22:%5B%2211%22%5D,%22mapScroll%22:false%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_only_houses_lisboa= pd.DataFrame(property_list)

C:\Users\roych\AppData\Local\Temp\ipykernel_7852\3788092327.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
df_buy_only_houses_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1520 non-null   object
 1   Address    1520 non-null   object
 2   Prop_type  1520 non-null   object
 3   Size       1520 non-null   object
 4   Bath       1520 non-null   object
 5   Bed        1520 non-null   object
dtypes: object(6)
memory usage: 71.4+ KB


In [10]:
df_buy_only_houses_lisboa.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,475 000 €,"Lisboa - Cascais, São Domingos de Rana",House,104 m2,3,3
1,650 000 €,"Lisboa - Oeiras, Barcarena",House,260 m2,4,5
2,4 500 000 €,"Lisboa - Lisboa, Belém",House,336 m2,5,5
3,190 000 €,"Lisboa - Alenquer, Alenquer (Santo Estêvão ...",House,146 m2,2,4
4,138 000 €,"Lisboa - Vila Franca de Xira, Vila Franca d...",House,59 m2,0,1


In [11]:
# code for getting "BUY" listings - CONDOS/APTS part 1 // Lisboa
# page only goes to 100, so cannot scrap all at once
# 0 < price < 250,000 - 1,454  properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 73 #661 houses for rent, last page is 73 (1,454 /20 = 72.7)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Lisboa%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2276%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:38.8404598668955,%22longitude%22:-9.2186952991834%7D,%22regionZoom%22:9,%22prn%22:%22Lisboa%22,%22listingClass%22:1,%22listingTypes%22:%5B%221%22%5D,%22mapScroll%22:false,%22price%22:%7B%22min%22:0,%22max%22:249999%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Con_Aprt_lessThan250_lisboa = pd.DataFrame(property_list)


C:\Users\roych\AppData\Local\Temp\ipykernel_7852\200980123.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [12]:
df_buy_Con_Aprt_lessThan250_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1460 non-null   object
 1   Address    1460 non-null   object
 2   Prop_type  1460 non-null   object
 3   Size       1460 non-null   object
 4   Bath       1460 non-null   object
 5   Bed        1460 non-null   object
dtypes: object(6)
memory usage: 68.6+ KB


In [13]:
df_buy_Con_Aprt_lessThan250_lisboa.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,135 000 €,"Lisboa - Vila Franca de Xira, Póvoa de Sant...",Condo/Apartment,50 m2,1,2
1,100 000 €,"Lisboa - Loures, Santa Iria de Azoia, São J...",Condo/Apartment,88 m2,1,1
2,165 000 €,"Lisboa - Oeiras, Carnaxide e Queijas",Condo/Apartment,52 m2,1,1
3,138 500 €,"Lisboa - Vila Franca de Xira, Póvoa de Sant...",Condo/Apartment,71 m2,1,2
4,198 000 €,"Lisboa - Amadora, Venteira",Condo/Apartment,97 m2,2,3


In [14]:
# code for getting for buy listings - CONDOS/APTS part 2 //lisboa
# page only goes to 100, so cannot scrap all at once
# 250,000 < price < 500,000 - 1,680 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 84 #1,680 houses for rent, last page is 84 (1,680/20 = 84)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Lisboa%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2276%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:38.8404598668955,%22longitude%22:-9.2186952991834%7D,%22regionZoom%22:9,%22prn%22:%22Lisboa%22,%22listingClass%22:1,%22listingTypes%22:%5B%221%22%5D,%22mapScroll%22:false,%22price%22:%7B%22min%22:250000,%22max%22:499999%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Con_Aprt_250K_500K_lisboa = pd.DataFrame(property_list)


C:\Users\roych\AppData\Local\Temp\ipykernel_7852\97184685.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [15]:
df_buy_Con_Aprt_250K_500K_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1680 non-null   object
 1   Address    1680 non-null   object
 2   Prop_type  1680 non-null   object
 3   Size       1680 non-null   object
 4   Bath       1680 non-null   object
 5   Bed        1680 non-null   object
dtypes: object(6)
memory usage: 78.9+ KB


In [16]:
df_buy_Con_Aprt_250K_500K_lisboa.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,385 000 €,"Lisboa - Cascais, Cascais e Estoril",Condo/Apartment,83 m2,1,2
1,485 000 €,"Lisboa - Cascais, Cascais e Estoril",Condo/Apartment,107 m2,2,3
2,395 000 €,"Lisboa - Lisboa, Penha de França",Condo/Apartment,125 m2,1,2
3,365 000 €,"Lisboa - Oeiras, Algés, Linda-a-Velha e Cru...",Condo/Apartment,121 m2,2,3
4,295 000 €,"Lisboa - Cascais, Carcavelos e Parede",Condo/Apartment,74 m2,1,2


In [17]:
# code for getting for buy listings - CONDS/APTS part 3 // lisboa
# page only goes to 100, so cannot scrap all at once
# price > 500,000 - 1,505 properties

driver = webdriver.Chrome(ChromeDriverManager().install())
property_list = []

last_page = 76 #1,505 houses for rent, last page is 76 (1,505/20 = 75.25)
pages = list(range(1, last_page+1))

url = 'https://www.remax.pt/en/buy?searchQueryState=%7B%22regionName%22:%22Lisboa%22,%22businessType%22:1,%22page%22:1,%22regionID%22:%2276%22,%22regionType%22:%22Region1ID%22,%22sort%22:%7B%22fieldToSort%22:%22ContractDate%22,%22order%22:1%7D,%22mapIsOpen%22:false,%22regionCoordinates%22:%7B%22latitude%22:38.8404598668955,%22longitude%22:-9.2186952991834%7D,%22regionZoom%22:9,%22prn%22:%22Lisboa%22,%22listingClass%22:1,%22listingTypes%22:%5B%221%22%5D,%22mapScroll%22:false,%22price%22:%7B%22min%22:500000,%22max%22:null%7D%7D'

for page in pages:
    url_page = url.format(page)
    driver.get(url_page)
    time.sleep(15)
    content = driver.page_source
    soup = BeautifulSoup(content, features='html.parser')

    for element in soup.findAll('div', attrs={'class': 'col-12 col-sm-6 col-md-6 col-lg-4 col-xl-3 result'}):
        prop_item ={}
        prop_item['Price'] = element.find('p', attrs={'class': 'listing-price'}).text     
        prop_item['Address'] = element.find('h2', attrs={'class': 'listing-address'}).text    
        try:
            prop_item['Prop_type'] = element.find('li', attrs={'class': 'listing-type'}).text
        except AttributeError: prop_item['Prop_type'] =""    
        prop_item['Size'] = element.find('li', attrs={'class': 'listing-area'}).text
        prop_item['Bath'] = element.find('li', attrs={'class': 'listing-bathroom'}).text
        prop_item['Bed'] = element.find('li', attrs={'class': 'listing-bedroom'}).text

        property_list.append(prop_item)
    
driver.quit()

df_buy_Aprt_Above_500K_lisboa = pd.DataFrame(property_list)


C:\Users\roych\AppData\Local\Temp\ipykernel_7852\3896805261.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
df_buy_Aprt_Above_500K_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      1520 non-null   object
 1   Address    1520 non-null   object
 2   Prop_type  1520 non-null   object
 3   Size       1520 non-null   object
 4   Bath       1520 non-null   object
 5   Bed        1520 non-null   object
dtypes: object(6)
memory usage: 71.4+ KB


In [19]:
df_buy_Aprt_Above_500K_lisboa.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,1 600 000 €,"Lisboa - Lisboa, Olivais",Condo/Apartment,198 m2,0,7
1,1 500 000 €,"Lisboa - Lisboa, Avenidas Novas",Condo/Apartment,187 m2,3,3
2,997 500 €,"Lisboa - Cascais, Cascais e Estoril",Condo/Apartment,136 m2,1,2
3,535 000 €,"Lisboa - Lisboa, Campolide",Condo/Apartment,130 m2,2,2
4,550 000 €,"Lisboa - Odivelas, Odivelas",Condo/Apartment,158 m2,2,3


<font face="Lucida Handwriting
" size=4 color='#CDB79E'> Connecting all the data into one 
    CSV file <font>


In [20]:
frames_lisboa = [df_buy_only_houses_lisboa, df_buy_Con_Aprt_lessThan250_lisboa,df_buy_Con_Aprt_250K_500K_lisboa, df_buy_Aprt_Above_500K_lisboa ]
df_buy_lisboa = pd.concat(frames_lisboa)
df_buy_lisboa.to_csv(r'lisboa_Buy.csv', index = False, header=True) #saving as csv

In [21]:
data_lisboa = pd.read_csv('lisboa_Buy.csv')

In [22]:
df_buy_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6180 entries, 0 to 1519
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      6180 non-null   object
 1   Address    6180 non-null   object
 2   Prop_type  6180 non-null   object
 3   Size       6180 non-null   object
 4   Bath       6180 non-null   object
 5   Bed        6180 non-null   object
dtypes: object(6)
memory usage: 338.0+ KB


In [23]:
df_buy_lisboa.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,475 000 €,"Lisboa - Cascais, São Domingos de Rana",House,104 m2,3,3
1,650 000 €,"Lisboa - Oeiras, Barcarena",House,260 m2,4,5
2,4 500 000 €,"Lisboa - Lisboa, Belém",House,336 m2,5,5
3,190 000 €,"Lisboa - Alenquer, Alenquer (Santo Estêvão ...",House,146 m2,2,4
4,138 000 €,"Lisboa - Vila Franca de Xira, Vila Franca d...",House,59 m2,0,1


In [28]:
frames_All = [data_Porto,data_Braga,df_buy_lisboa]
df_buy_All = pd.concat(frames_All)
df_buy_All.to_csv(r'Real_estate_data_Portugal.csv', index = False, header=True) #saving as csv

<font face="Lucida Handwriting
" size=4 color='#CDB79E'> Connecting all the data into one 
    CSV file <font>


In [29]:
pd = pd.read_csv('Real_estate_data_Portugal.csv')   

In [30]:
pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11340 entries, 0 to 11339
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Price      11340 non-null  object
 1   Address    11340 non-null  object
 2   Prop_type  11280 non-null  object
 3   Size       11340 non-null  object
 4   Bath       11340 non-null  object
 5   Bed        11340 non-null  object
dtypes: object(6)
memory usage: 531.7+ KB


In [31]:
pd.head()

,Price,Address,Prop_type,Size,Bath,Bed
0,350 000 €,"Porto - Maia, Castêlo da Maia",House,233 m2,4,4
1,98 000 €,"Porto - Paredes, Lordelo",House,60 m2,1,2
2,300 000 €,"Porto - Felgueiras, Vila Fria e Vizela (São...",House,642 m2,3,4
3,199 900 €,"Porto - Amarante, Telões",House,188 m2,3,3
4,235 000 €,"Porto - Marco de Canaveses, Tabuado",House,200 m2,2,3
